# Incorporating corrosion design constraints in WaterTAP
This tutorial demonstrates how to build surrogate models for general and localized corrosion design constraints and how to integrate these surrogate models in a WaterTAP model of MVC treating seawater (Tucker et al., 'Incorporating corrosion design constraints in desalination process optimization.' In preparation).

There are three primary steps to developing surrogate models for those constraints and integrating them: 
1. Data generation of corrosion metrics
2. Fitting corrosion surrogate models
3. Integrating corrosion design constraints

![](corrosion_demo_figures\methods_figure.png)

## 1. Data generation of corrosion metrics
For a material to be considered corrosion resistant, we assume the following two constraints must be satisfied:
1. Corrosion rate is less than a maximum corrosion rate of 0.1 mm/yr (sufficiently low general corrosion): $$CR \leq CR_{max}$$
2. The repassivation potential is greater than the corrosion potential (conservative prediction of no localized corrosion): $$V_{rp}-V_{c} \geq 0$$

We used [OLI Systems' Corrosion Analyzer](https://www.olisystems.com/software/oli-studio/oli-studio-corrosion-analyzer/) to predict corrosion rate $CR$, repassivation potential $V_{rp}$, and corrosion potential $V_{c}$ data under different operating conditions relevant to the evaporator in MVC. 

We considered the following ranges of inputs:
* Temperature: 25-95 C (15 samples)
* pH: 4-8 (9 samples)
* Recovery: 0-80% (17 samples)
* Dissolved oxygen: 0-8 mg/L (9 samples)
* Materials: carbon steel 1018 → stainless steel 304 → stainless steel 316 → duplex stainless steel 2205 → duplex stainless steel 2507 → nickel alloy 825 → nickel alloy 625 (7 materials listed in order of increasing cost)

We decided on these ranges based on the application of MVC but also through initial exploration: 

<img src="corrosion_demo_figures\material_selection.png" alt="material_selection" width="624"/>

This script for data generation via [OLI's Cloud API](https://www.olisystems.com/software/oli-cloud-apis/) is included but requires an OLI license to run. 

<img src="corrosion_demo_figures\oli_api_call.png" alt="oli_api" width="624"/>

### 1.1 Data for the demo
For the purpose of this demo, we will use data that has been generated as a function of temperature and dissolved oxygen for the repassivation-corrosion potential difference to predict localized corrosion. 

<img src="corrosion_demo_figures\synthetic_material_behavior.png" alt="synthetic_material_behavior" width="624"/>

The code used to generate and plot the synthetic data can be found in the 'week7/synthetic_corrosion_data' folder.

## 2. Fitting corrosion surrogate models
We use an adaptive sampling approach and [PySMO](https://idaes-pse.readthedocs.io/en/1.5.1/surrogate/pysmo/index.html) to fit [radial basis functions](https://idaes-pse.readthedocs.io/en/1.5.1/surrogate/pysmo/pysmo_radialbasisfunctions.html) with a cubic basis for the corrosion rate and repassivation corrosion potential difference for each material. For our application, RBFs work well for representing highly nonlinear relationships and modeling diverse operating regimes. 

We first select an initial training size and use [Hammersley sampling](https://idaes-pse.readthedocs.io/en/1.5.1/surrogate/pysmo/pysmo_hammersley.html) to select initial training set. We then add additional samples to improve the model fit. For fitting the general corrosion surrogate, we add the samples with the worst absolute error. For the localized corrosion surrogate, we add samples in two phases. In the first phase, we consider the set of misclassified samples and add the samples with the worst absolute error to improve the classification accuracy near 0 V for localized corrosion. In the second phase, we add the samples with the worst absolute error to minimize the overall error. 

We show below how to fit on the generated data.

In [1]:
%%capture
from pathlib import Path
import pandas as pd
import fit_surrogates

current_directory = Path.cwd()
folder = current_directory.parent / f"week7/synthetic_corrosion_data/"

survey_path = current_directory.parent / f"week7/synthetic_corrosion_data/synthetic_potential_difference.csv"
data = pd.read_csv(survey_path)
output = 'synthetic_potential_difference_V'

# Adaptive sampling parameters
n = (10, # initial number of samples
     3, # number of iterations of adding worst misclassified samples
     3, # number of iterations of adding worst error samples
     5) # number of samples to add per iteration

# create surrogate object
basis = 'cubic'
sf = fit_surrogates.surrogateFitting(data=data,
                          input_labels=['temperature_C','do_mg_L'],
                          output_labels=[output],
                          n_init=n[0],
                          n_mid=n[1],
                          n_final=n[2],
                          n_add=n[3],
                          initial_sample_type='Hammersley',
                          basis=basis)

# Initial fit
sf.fit_surrogate(output)

# Add points based on worst misclassified
for i in range(sf.n_mid):
    sf.add_training_sample(type='worst_misclassified', output=output)
    sf.fit_surrogate(output)

# Add points based on worst error
for i in range(sf.n_final):
    sf.add_training_sample(type='worst_error', output=output)
    sf.fit_surrogate(output)

# save final surrogate
surrogate_file = folder / f'{output}_pysmo_rbf_{basis}_surrogate.json'
model = sf.surrogates[output]['surrogate'].save_to_file(surrogate_file, overwrite=True)

<img src="corrosion_demo_figures\synthetic_potential_difference_V_10_3_3_5_cubic_error_vs_iteration.png" alt="adaptive_sampling" width="400"/>

### Assess the error metrics.
We want the maximum absolute error (maxAE) to be less than 0.05 V (SHE), the misclassification rate less than 1%, and the balanced accuracy greater than 99%. 

In [2]:
# print the error metrics
errors = ['MSE', 'R2', 'maxAE', 'misclassification', 'balanced_accuracy']
for err in errors:
    print(f'{err}: {str(round(sf.surrogates[output][err][-1], 4))}') # final error

MSE: 0.0003
R2: 0.9836
maxAE: 0.0565
misclassification: 1.5686
balanced_accuracy: 97.9381


## 3. Integrating corrosion design constraints
We will first build and solve the [MVC flowsheet](https://github.com/watertap-org/watertap/tree/main/watertap/flowsheets/mvc) without corrosion and then add the corrosion surrogates for a chosen material. In this demo, we will assume the generated data approximates Duplex Stainless Steel 2507. We will compare the results of without and with corrosion design constraints for the case of 70 g/kg to 58% recovery.

### Import modules and MVC flowsheet

In [3]:
import mvc_corrosion as mvc
from watertap.core.solvers import get_solver
from pyomo.environ import (Objective, Var, Constraint, assert_optimal_termination, value, units as pyunits)
from pyomo.util.calc_var_value import calculate_variable_from_constraint
from idaes.core.util.model_statistics import degrees_of_freedom
import idaes.core.util.scaling as iscale
from idaes.core.surrogate.pysmo_surrogate import PysmoSurrogate
from idaes.core.surrogate.surrogate_block import SurrogateBlock

### Build and solve MVC flowsheet without corrosion
The key decision variables are the evaporator temperature, evaporator area, compressor pressure ratio, and heat exchanger areas.

In [4]:
material = 'Duplex stainless 2205'

m = mvc.build(material=material)
mvc.set_operating_conditions(m)
mvc.add_Q_ext(m, time_point=m.fs.config.time)
mvc.initialize_system(m)
mvc.scale_costs(m)
mvc.fix_outlet_pressures(m)  # outlet pressure are initially unfixed for initialization

# set up for minimizing Q_ext in first solve - should be 1 DOF because Q_ext is unfixed
print("DOF after initialization: ", degrees_of_freedom(m))
m.fs.objective = Objective(expr=m.fs.Q_ext[0])

# First solve - minimizing external Q
solver = get_solver()
results = mvc.solve(m, solver=solver, tee=False)
print("First solve termination condition: ", results.solver.termination_condition)
assert_optimal_termination(results)

# Second solve with LCOW optimization - conditions of 100 g/kg, 50% recovery
mvc.add_evap_hx_material_factor_equal_constraint(m)
m.fs.Q_ext[0].fix(0)  # no longer want external heating in evaporator
del m.fs.objective
mvc.set_up_optimization(m)
results = mvc.solve(m, solver=solver, tee=False)
print("Second solve termination condition: ", results.solver.termination_condition)

# Third solve with LCOW optimization - conditions of 70 g/kg, 58% recovery (base case)
rr = 0.58
wf = 70 
m.fs.recovery[0].fix(rr)
m.fs.feed.properties[0].mass_frac_phase_comp["Liq", "TDS"].fix(wf/1e3)
m.fs.costing.evaporator.material_factor_cost.fix(6.5)
results = mvc.solve(m, solver=solver, tee=False)
print("Third solve termination condition: ", results.solver.termination_condition)
lcow_original = value(m.fs.costing.LCOW)
temp_evap_original = value(m.fs.evaporator.properties_brine[0].temperature)

DOF after setting operating conditions:  0
Initialization termination condition:  optimal
Scaled costs
DOF after initialization:  1
First solve termination condition:  optimal
DOF for optimization:  4
Second solve termination condition:  optimal
Third solve termination condition:  optimal


### Add corrosion constraints
1. Add dimensionless input variables for corrosion surrogates: temperature, brine salinity, dissolved oxygen, and pH. For this tutorial, only temperature and dissolved oxygen are used as inputs. 
2. Add dimensionless output variables: general corrosion rate (corrosion_rate) and repassivation-corrosion potential difference (potential_difference). For this tutorial, only the repassivation-corrosion potential difference is added. 
3. Load fitted surrogates

In [5]:
def add_localized_corrosion_surrogate(m):
    # Temperature - surrogate input is in C
    m.fs.temperature_indexed = Var([0],
                                   initialize=m.fs.evaporator.properties_brine[0].temperature.value,
                                   units=pyunits.dimensionless) # 
    m.fs.eq_temperature_indexed = Constraint(
            expr=m.fs.evaporator.properties_brine[0].temperature == m.fs.temperature_indexed[0] + 273.15
        )
    # Brine salinity
    brine_salt = m.fs.evaporator.properties_brine[0].flow_mass_phase_comp['Liq','TDS'].value
    brine_water = m.fs.evaporator.properties_brine[0].flow_mass_phase_comp['Liq','H2O'].value
    m.fs.brine_salinity_indexed = Var([0],
                                      initialize=brine_salt/(brine_water + brine_salt),
                                      units=pyunits.dimensionless)
    m.fs.eq_brine_salinity_indexed = Constraint(
        expr=m.fs.evaporator.properties_brine[0].mass_frac_phase_comp['Liq','TDS'] == m.fs.brine_salinity_indexed[0]
        )
    # Dissolved oxygen - fixed
    m.fs.dissolved_oxygen_index = Var([0],
                                      initialize=0,
                                      units=pyunits.dimensionless,
                                      bounds=(0,8))
    # pH - fixed
    m.fs.pH_index = Var([0],
                        initialize=7.5,
                        units=pyunits.dimensionless,
                        bounds=(4, 8))
    m.fs.pH_index.fix()
    
    # Repassivation-corrosion potential difference
    m.fs.potential_difference_indexed = Var([0],
                                            initialize=0.0,
                                            units=pyunits.dimensionless)
    iscale.set_scaling_factor(m.fs.potential_difference_indexed[0], 1e3)
    
    m.fs.potential_difference = Var(initialize=0,
                                    bounds=(0,100), # cannot be less than 0 V
                                    units=pyunits.V)
    iscale.set_scaling_factor(m.fs.potential_difference, 1e3)
    
    m.fs.eq_potential_difference_indexed = Constraint(
        expr=m.fs.potential_difference == m.fs.potential_difference_indexed[0]
    )

    # Add localized corrosion surrogate - input order:'temperature_C','dissolved_oxygen_mg_L'
    filename = folder / "synthetic_potential_difference_V_pysmo_rbf_cubic_surrogate.json"
    potential_difference_surrogate = PysmoSurrogate.load_from_file(filename)
    m.fs.potential_difference_surrogate = SurrogateBlock(concrete=True)
    m.fs.potential_difference_surrogate.build_model(potential_difference_surrogate,
                                              input_vars=[m.fs.temperature_indexed[0],
                                                          m.fs.dissolved_oxygen_index[0]],
                                              output_vars=[m.fs.potential_difference_indexed[0]])
    # check value
    calculate_variable_from_constraint(m.fs.potential_difference_indexed[0], m.fs.potential_difference_surrogate.pysmo_constraint['synthetic_potential_difference_V'])

### Fix the corrosion conditions
We fix the level of dissolved oxygen.

In [6]:
add_localized_corrosion_surrogate(m)
do = 0.5 # assume almost all dissolved oxygen as been removed
m.fs.dissolved_oxygen_index[0].fix(do)


Default parameter estimation method is used.

Parameter estimation method:  algebraic
Basis function:  cubic
Regularization done:  True


### Update material factor based on material selected

In [7]:
material_factor = {
        "Carbon steel 1018": 1,
        "Stainless steel 304": 3.0,
        "Stainless steel 316": 3.2,
        "Duplex stainless 2205": 3.5,
        "Duplex stainless 2507": 4.0,
        "Nickel alloy 825": 5.0,
        "Nickel alloy 625": 6.0
    }
m.fs.costing.evaporator.material_factor_cost.fix(material_factor[m.fs.material.value])
print(f'Evaporator material factor: {m.fs.costing.evaporator.material_factor_cost.value}')

Evaporator material factor: 3.5


### Update evaporator temperature bounds
When we were not considering corrosion, the evaporator temperature upper bound was 75 C. 

In [8]:
m.fs.evaporator.properties_brine[0].temperature.setub(95 + 273.15)

### Solve model with corrosion constraints
Solving for a feed concentration of 70 g/kg and recovery of 58%.

In [9]:
results = mvc.solve(m, solver=solver, tee=False)
print("Termination condition: ", results.solver.termination_condition)
mvc.display_demo(m)

Termination condition:  optimal
Levelized cost of water:                  3.76 $/m3
Evaporator (brine, vapor) temperature:    74.89 C
Evaporator material factor:               3.50 
Dissolved oxygen:                         0.50 mg/L
Potential difference:                     0.0000 V


### Increase the level of dissolved oxygen from 0.5 to 8 mg/L
The result will be infeasible because duplex stainless steel 2507 has poor resistance to localized corrosion at high levels of dissolved oxygen.

In [10]:
m.fs.dissolved_oxygen_index[0].fix(8) # increase to 8 mg/L
results = mvc.solve(m, solver=solver, tee=False)
print("Termination condition: ", results.solver.termination_condition)

The current configuration is infeasible. Please adjust the decision variables.
Termination condition:  infeasible


### How do we know that this due to the localized corrosion bound?
Remove the bound on potential difference. 

In [11]:
m.fs.potential_difference.setlb(None)
results = mvc.solve(m, solver=solver, tee=False)
print("Termination condition: ", results.solver.termination_condition)
mvc.display_demo(m)

Termination condition:  optimal
Levelized cost of water:                  3.70 $/m3
Evaporator (brine, vapor) temperature:    95.00 C
Evaporator material factor:               3.50 
Dissolved oxygen:                         8.00 mg/L
Potential difference:                     -0.1920 V


## Results without vs. with corrosion
For all levels of dissolved oxygen, we have the same result when not accounting for corrosion. At low levels of dissolved oxygen, we have overestimated the cost while at high levels of dissolved oxygen we are at a similar cost but do not know if the operating conditions and material selection are corrosion resistant.
<img src="demo_figures\synthetic_lcow_comparison.png" alt="lcow_comparison" width="312"/>

## Sensitivity to dissolved oxygen
Using the [parameter sweep tool](https://watertap.readthedocs.io/en/stable/how_to_guides/how_to_use_parameter_sweep.html#how-to-explore-a-model-with-parameter-sweep), we solve for the cost-optimal design and operation for each material at different levels of dissolved oxygen for the case of 70 g/kg to 58%. Going from 8 mg/L to 0.5 mg/L reduces the LCOW by 1.16 $/m3. 

<img src="demo_figures\do_sensitivity.png" alt="do_sensitivity" width="312"/>

## Cost-optimal design across feed concentrations and recoveries
Using the parameter sweep tool, we can find the cost-optimal material that prevents corrosion across a range of feed concentrations and recoveries.
<img src="demo_figures\cost_optimal_panel.png" alt="cost_optimal_panel" width="624"/>

# Conclusions
We have fit a surrogate to represent localized corrosion, and we are able to account for corrosion design constraints in selecting materials for the evaporator in MVC. For other applications, other input variables and factors affecting material selection can be incorporated. 

Surrogate model integration is effective when:
* your mechanistic model is too complex to represent in your process optimization model.
* you have access to data or a simulator that can be used to train a surrogate model for your specific case. 